# 250616.1

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# For digit classification
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        # For nn.Conv2d definition (see 250616.3) 
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # For why Linear layers are used after Convolutional layers (see 250618.4)
        # nn.Linear represent an affine operation: y = Wx + b. For for nn.Linear definition (see 250617.1)  
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # outputs a vector size of 10, because it for classifying 10 digits.

    def forward(self, input):
        
        # Convolution layer C1: 1 input image channel (means grayscale image - see 250616.4), 6 output channels (see 250616.5),
        # 5x5 square convolution, it uses RELU activation function. 
        # For why RELU is needed after each layer (see 250618.6)
        # Output: a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        # For definition of a subsampling / max_pool2d layer (see 250618.5)
        s2 = F.max_pool2d(c1, (2, 2))
        
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        # Turns a matrix into a vector. nn.Linear only works on vectors
        s4 = torch.flatten(s4, 1)
        
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))

        # OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        return output

    # only need to define forward, backward function is automatically defined using autograd


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

# Q: what do these negative and positive numbers mean? how do i figure out 

tensor([[-0.0181, -0.0806,  0.0819,  0.0443,  0.1519, -0.0408,  0.0105, -0.0321,
          0.1161,  0.0819]], grad_fn=<AddmmBackward0>)


In [3]:
# Q: what is zero_grad doing here?

# Q: what is backward doing here?

net.zero_grad()
out.backward(torch.randn(1, 10))